In [4]:
!pip install pandas

In [5]:
import pandas as pd
my_file_path = r"C:\Users\user\Desktop\iit\year2 sem1\sdgp\AgroEdge\farming data2.csv"
farming_data=pd.read_csv(my_file_path)
farming_data.head()

,Farmer Name,Farm ID,Farm Size (Perches),Farm Type,Farm Location (Village),Farm Location (District),Crop Type,Brinjal Variety,Planting Start Date,Planting End Date,...,Total Water Usage (liters),Irrigation Schedule,Irrigation Cost (Rs.),Total Number of Labor Hours,Total Labor Wages,Market Selling Price (Rs./kg),Total Sales Revenue (Rs.),Profit/Loss (Rs.),Buyer Type,Storage Method
0,Eran Wickramasinghe,6710,9.0,Organic,Hanwella,Colombo,Brinjal,Local,27/10/2024,20/01/2025,...,159219.33,Daily,4045.21,8,2453.45,154.11,40680.42,4040.16,Retail,Refrigeration
1,Chandana Fernando,7527,112.0,Mixed,Minuwangoda,Gampaha,Brinjal,Other,07/05/2024,28/08/2024,...,1505123.64,Weekly,1735.96,8,1399.81,100.71,27617.70,8243.59,Retail,Refrigeration
2,Amara Perera,4828,182.4,Conventional,Bulathsinhala,Kalutara,Brinjal,Hybrid,27/07/2024,16/11/2024,...,2481867.21,Daily,3469.49,9,2561.95,371.97,109366.62,76595.58,Direct Consumers,Refrigeration
3,Chandana Fernando,5629,352.0,Mixed,Pussellawa,Kandy,Brinjal,Hybrid,23/06/2023,04/10/2023,...,4903479.39,Based on moisture levels,643.28,5,2609.61,444.54,87294.32,67207.80,Retail,Refrigeration
4,Dinesh Kumara,5543,441.6,Mixed,Rattota,Matale,Brinjal,Other,14/03/2023,21/05/2023,...,7335547.72,Based on moisture levels,3593.12,8,2025.32,313.02,139995.06,111829.22,Wholesale,Refrigeration


In [6]:
print(farming_data.isnull().sum())

Farmer Name                         0
Farm ID                             0
Farm Size (Perches)                 0
Farm Type                           0
Farm Location (Village)             0
Farm Location (District)            0
Crop Type                           0
Brinjal Variety                     0
Planting Start Date                 0
Planting End Date                   0
Expected Harvest (kg)               0
Actual Harvest (kg)                 0
Soil Type                           0
Soil pH Level                       1
Seed Type                           0
Seed Variety                        0
Seed Source                         0
Seed Quantity (g)                   0
Seed Cost (Rs.)                     0
Fertilizer Type                     0
Fertilizer Brand                    0
Total Fertilizer Quantity (kg)      0
Fertilizer N-P-K Ratio              0
Fertilizer Application Schedule     0
Fertilizer Application Method       0
Fertilizer Cost (Rs.)               0
Pesticide Ty

In [7]:
import numpy as np
from scipy.stats import zscore

logical_min_threshold = 10 #asume the smallest value that can be farmsize

# Compute IQR (Interquartile Range)
Q1 = farming_data['Farm Size (Perches)'].quantile(0.25)  # 25th percentile
Q3 = farming_data['Farm Size (Perches)'].quantile(0.75)  # 75th percentile

IQR = Q3 - Q1   #calculate IQR

upper_bound_iqr= Q3 + 1.2 * IQR # Compute an upper bound dynamically

#calculate zcore
z_scores = zscore(farming_data['Farm Size (Perches)']) 

# Identify outliers based on above criteria
outlier_indices = farming_data.index[
    (farming_data['Farm Size (Perches)'] < logical_min_threshold) |  # Very small values
    (farming_data['Farm Size (Perches)'] > upper_bound_iqr) |        # IQR-based high outliers
    (z_scores > 2.5)                                                 # Z-score-based extreme values
]
removed_outliers = farming_data.loc[outlier_indices, ['Farm Size (Perches)']]
#remove those outliers 
farming_data.loc[outlier_indices, 'Farm Size (Perches)'] = np.nan
print(removed_outliers)


    Farm Size (Perches)
0                   9.0
11                  8.0
12                  7.0
49                  8.0
59                  7.0
75                  6.0
85                  9.0


In [8]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

# Avoid division errors (if Farm Size is 0 or NaN)
valid_farm_data = farming_data[farming_data['Farm Size (Perches)'] > 0].copy()

# Compute Expected Yield per Perch dynamically
expected_yield_per_perch = valid_farm_data['Expected Harvest (kg)'] / valid_farm_data['Farm Size (Perches)']

# Ensure index matches with the original dataset
expected_yield_per_perch = expected_yield_per_perch.reindex(farming_data.index, fill_value=np.nan)

# Compute IQR for 'Expected Yield per Perch'
Q1 = expected_yield_per_perch.quantile(0.25)
Q3 = expected_yield_per_perch.quantile(0.75)
IQR = Q3 - Q1

# Compute IQR-based limits
upper_bound_iqr = Q3 + 1.5 * IQR  # Upper threshold
lower_bound_iqr = max(Q1 - 1.5 * IQR, 0.1)  # Lower threshold (avoiding zero)

# Compute Z-score for extreme high values dynamically
z_scores = np.abs(zscore(expected_yield_per_perch, nan_policy="omit"))

# Identify Outliers based on Yield per Perch
outlier_mask = (
    (expected_yield_per_perch < lower_bound_iqr) | 
    (expected_yield_per_perch > upper_bound_iqr) |
    (z_scores > 3)  # Extreme high values only
)

# Extract and print the detected outliers (Now the index matches)
outliers_df = farming_data.loc[outlier_mask, ['Farm Size (Perches)', 'Expected Harvest (kg)']]
print("Detected Unrealistic Expected Harvest Values (Based on Farm Size):")
print(outliers_df)

# Replace only outlier values with NaN (keeping the row intact)
farming_data.loc[outlier_mask, 'Expected Harvest (kg)'] = np.nan


Detected Unrealistic Expected Harvest Values (Based on Farm Size):
Empty DataFrame
Columns: [Farm Size (Perches), Expected Harvest (kg)]
Index: []


In [9]:
# Ensure we have no zero farm sizes to avoid division errors
valid_farm_data = farming_data[farming_data['Farm Size (Perches)'] > 0].copy()

# Compute "Actual Yield per Perch" dynamically (without adding a new column)
actual_yield_per_perch = valid_farm_data['Actual Harvest (kg)'] / valid_farm_data['Farm Size (Perches)']

# Reindex to match original farming_data to avoid indexing errors
actual_yield_per_perch = actual_yield_per_perch.reindex(farming_data.index, fill_value=np.nan)

# Compute IQR for "Actual Yield per Perch"
Q1 = actual_yield_per_perch.quantile(0.25)  # 25th percentile
Q3 = actual_yield_per_perch.quantile(0.75)  # 75th percentile
IQR = Q3 - Q1  # Interquartile range (spread of the middle 50% of data)

# Set thresholds for identifying outliers using IQR
upper_bound_iqr = Q3 + 1.5 * IQR  # Any value above this is considered an outlier
lower_bound_iqr = max(Q1 - 1.5 * IQR, 0.1)  # Any value below this is also an outlier (but avoid zero or negative values)

# Compute Z-score for extreme values dynamically
z_scores = np.abs(zscore(actual_yield_per_perch, nan_policy="omit"))

# Identify outliers based on "Actual Yield per Perch"
outlier_mask = (
    (actual_yield_per_perch < lower_bound_iqr) | 
    (actual_yield_per_perch > upper_bound_iqr) |  
    (z_scores > 3)  # Extreme outliers (Z-score greater than 3)
)

# Extract detected outliers
outliers_df = farming_data.loc[outlier_mask, ['Farm Size (Perches)', 'Actual Harvest (kg)']]

# Display detected outliers using standard pandas display
print("\nDetected Outliers in Actual Harvest:")
print(outliers_df)

# Replace only outlier values with NaN (keeping the row intact)
farming_data.loc[outlier_mask, 'Actual Harvest (kg)'] = np.nan


Detected Outliers in Actual Harvest:
Empty DataFrame
Columns: [Farm Size (Perches), Actual Harvest (kg)]
Index: []


In [10]:
# Ensure valid farm sizes to prevent division errors
valid_farm_data = farming_data[farming_data['Farm Size (Perches)'] > 0].copy()

# Compute "Seed Usage per Perch" dynamically (without modifying dataset)
seed_usage_per_perch = valid_farm_data['Seed Quantity (g)'] / valid_farm_data['Farm Size (Perches)']

# Reindex to match original farming_data
seed_usage_per_perch = seed_usage_per_perch.reindex(farming_data.index, fill_value=np.nan)

# Compute IQR for "Seed Usage per Perch"
Q1 = seed_usage_per_perch.quantile(0.25)
Q3 = seed_usage_per_perch.quantile(0.75)
IQR = Q3 - Q1

# Adjust outlier detection criteria
upper_bound_iqr = Q3 + 2 * IQR  # Increased IQR multiplier
lower_bound_iqr = max(Q1 - 2 * IQR, 0.5)  # More relaxed lower bound

# Adjust Z-score threshold
z_score_threshold = 3.5  # Less strict than 3

# Compute Z-score
z_scores = np.abs(zscore(seed_usage_per_perch, nan_policy="omit"))

# Identify outliers
outlier_mask = (
    (seed_usage_per_perch < lower_bound_iqr) |
    (seed_usage_per_perch > upper_bound_iqr) |
    (z_scores > z_score_threshold)
)

# Extract and display detected outliers
outliers_df = farming_data.loc[outlier_mask, ['Farm Size (Perches)', 'Seed Quantity (g)']]
print("Updated Detected Outliers in Seed Quantity (g):")
print(outliers_df)

# Replace only outlier values with NaN (keeping the row intact)
farming_data.loc[outlier_mask, 'Seed Quantity (g)'] = np.nan


Updated Detected Outliers in Seed Quantity (g):
    Farm Size (Perches)  Seed Quantity (g)
21                580.8                  5
25                796.8                 50
28                 10.0              10000


In [11]:
# Ensure valid farm sizes to prevent division errors
valid_farm_data = farming_data[farming_data['Farm Size (Perches)'] > 0].copy()

# Compute "Water Usage per Perch" dynamically (without modifying dataset)
water_usage_per_perch = valid_farm_data['Total Water Usage (liters)'] / valid_farm_data['Farm Size (Perches)']

# Reindex to match original farming_data
water_usage_per_perch = water_usage_per_perch.reindex(farming_data.index, fill_value=np.nan)

# Compute IQR for "Water Usage per Perch"
Q1 = water_usage_per_perch.quantile(0.25)
Q3 = water_usage_per_perch.quantile(0.75)
IQR = Q3 - Q1

# Compute IQR-based outlier limits
upper_bound_iqr = Q3 + 1.5 * IQR  # Upper threshold
lower_bound_iqr = max(Q1 - 1.5 * IQR, 10)  # Lower threshold (avoiding zero or too small values)

# Compute Z-score for extreme values
z_scores = np.abs(zscore(water_usage_per_perch, nan_policy="omit"))

# Identify outliers based on "Water Usage per Perch"
outlier_mask = (
    (water_usage_per_perch < lower_bound_iqr) |  # Too low
    (water_usage_per_perch > upper_bound_iqr) |  # Too high
    (z_scores > 3)  # Extreme outliers
)

# Extract detected outliers
outliers_df = farming_data.loc[outlier_mask, ['Farm Size (Perches)', 'Irrigation Method', 'Total Water Usage (liters)']]

# Print detected outliers
print("Detected Outliers in Total Water Usage (liters):")
print(outliers_df)

# Replace only outlier values with NaN (keeping the row intact)
farming_data.loc[outlier_mask, 'Total Water Usage (liters)'] = np.nan

Detected Outliers in Total Water Usage (liters):
    Farm Size (Perches) Irrigation Method  Total Water Usage (liters)
55                192.0         Sprinkler                     10000.0
66                291.2         Sprinkler                 100000000.0
84                321.6  Flood Irrigation                  50000000.0


In [12]:
import pandas as pd
# Define the updated function with rounding to two decimal places
def fill_soil_ph(row):
    if pd.isnull(row['Soil pH Level']):  
        # Filter data for the same village
        village_data = farming_data[farming_data['Farm Location (Village)'] == row['Farm Location (Village)']]
        
        # Check if the village has at least 2 records with valid data
        if village_data['Soil pH Level'].count() >= 2:
            village_mean = round(village_data['Soil pH Level'].mean(skipna=True), 2)
            print(f"Filling missing value using village mean ({row['Farm Location (Village)']}): {village_mean}")
            return village_mean
        
        # If no valid data in the village, check the district
        district_data = farming_data[farming_data['Farm Location (District)'] == row['Farm Location (District)']]
        
        # Check if the district has at least 2 records with valid data
        if district_data['Soil pH Level'].count() >= 2:
            district_mean = round(district_data['Soil pH Level'].mean(skipna=True), 2)
            print(f"Filling missing value using district mean ({row['Farm Location (District)']}): {district_mean}")
            return district_mean
        
        # If neither village nor district has at least 2 valid records, return the global mean
        global_mean = round(farming_data['Soil pH Level'].mean(skipna=True), 2)
        print(f"Filling missing value using global mean: {global_mean}")
        return global_mean
    
    return row['Soil pH Level']  

# Apply function to fill missing values
farming_data['Soil pH Level'] = farming_data.apply(fill_soil_ph, axis=1)

# Check if any NaN values remain
missing_values = farming_data['Soil pH Level'].isnull().sum()

# Print the total number of missing values left
print(f"Total missing values remaining: {missing_values}")

Filling missing value using district mean (Kandy): 5.86
Total missing values remaining: 0


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
# Select relevant features for predicting Farm Size (Perches)
features = ['Expected Harvest (kg)', 'Actual Harvest (kg)', 'Seed Quantity (g)',
            'Total Fertilizer Quantity (kg)', 'Fertilizer Cost (Rs.)', 
            'Total Pesticide Quantity (kg)', 'Total Water Usage (liters)', 
            'Total Sales Revenue (Rs.)']

# Separate rows with and without missing values in "Farm Size (Perches)"
train_data = farming_data.dropna(subset=['Farm Size (Perches)'])
test_data = farming_data[farming_data['Farm Size (Perches)'].isna()]

# Prepare training data
X_train = train_data[features]
y_train = train_data['Farm Size (Perches)']

# Handle missing values in features by filling with median (for training)
X_train.fillna(X_train.median(), inplace=True)

# Train a regression model (Random Forest for better prediction)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prepare test data
X_test = test_data[features]
X_test.fillna(X_test.median(), inplace=True)

# Predict missing Farm Sizes
predicted_farm_sizes = model.predict(X_test)

# Fill missing values with predicted values
farming_data.loc[farming_data['Farm Size (Perches)'].isna(), 'Farm Size (Perches)'] = predicted_farm_sizes.round(2)

# Extract the rows where "Farm Size (Perches)" was previously missing
predicted_farm_sizes_df = farming_data.loc[test_data.index, ['Farm Size (Perches)']]

# Print the predicted values with corresponding farmer details
print("Predicted Farm Size (Perches) for Previously Missing Values:")
print(predicted_farm_sizes_df)

C:\Users\user\AppData\Local\Temp\ipykernel_36392\399748675.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.fillna(X_train.median(), inplace=True)


Predicted Farm Size (Perches) for Previously Missing Values:
    Farm Size (Perches)
0                 24.35
11                39.07
12                56.59
49                30.04
59                19.04
75                38.50
85                31.53


C:\Users\user\AppData\Local\Temp\ipykernel_36392\399748675.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(X_test.median(), inplace=True)


In [14]:
# Select relevant features for predicting Expected Harvest (kg)
features = ['Farm Size (Perches)', 'Seed Quantity (g)', 'Total Fertilizer Quantity (kg)',
            'Fertilizer Cost (Rs.)', 'Total Pesticide Quantity (kg)', 
            'Total Water Usage (liters)', 'Total Sales Revenue (Rs.)']

# Separate rows with and without missing values in "Expected Harvest (kg)"
train_data = farming_data.dropna(subset=['Expected Harvest (kg)'])
test_data = farming_data[farming_data['Expected Harvest (kg)'].isna()]

# Prepare training data
X_train = train_data[features]
y_train = train_data['Expected Harvest (kg)']

# Handle missing values in features by filling with median (for training)
X_train.fillna(X_train.median(), inplace=True)

# Train a regression model (Random Forest for better prediction)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prepare test data
X_test = test_data[features]
X_test.fillna(X_test.median(), inplace=True)

# Predict missing Expected Harvest values
predicted_expected_harvest = model.predict(X_test)

# Fill missing values with predicted values (rounded to 2 decimal places)
farming_data.loc[farming_data['Expected Harvest (kg)'].isna(), 'Expected Harvest (kg)'] = np.round(predicted_expected_harvest, 2)

# Extract the filled values for reference
predicted_harvest_df = farming_data.loc[test_data.index, ['Expected Harvest (kg)']]

# Print predicted values for missing Expected Harvest
print("Predicted Expected Harvest (kg) for Previously Missing Values:")
print(predicted_harvest_df)

C:\Users\user\AppData\Local\Temp\ipykernel_36392\2043593444.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.fillna(X_train.median(), inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_36392\2043593444.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(X_test.median(), inplace=True)


ValueError: Found array with 0 sample(s) (shape=(0, 7)) while a minimum of 1 is required by RandomForestRegressor.

In [ ]:
# Select relevant features for predicting Actual Harvest (kg)
features = ['Farm Size (Perches)', 'Expected Harvest (kg)', 'Seed Quantity (g)',
            'Total Fertilizer Quantity (kg)', 'Fertilizer Cost (Rs.)', 
            'Total Pesticide Quantity (kg)', 'Total Water Usage (liters)', 
            'Total Sales Revenue (Rs.)']

# Check for missing values in Actual Harvest (kg)
missing_count = farming_data['Actual Harvest (kg)'].isna().sum()
print("Missing values count in 'Actual Harvest (kg)':", missing_count)

if missing_count > 0:
    # Separate rows with and without missing values in "Actual Harvest (kg)"
    train_data = farming_data.dropna(subset=['Actual Harvest (kg)'])
    test_data = farming_data[farming_data['Actual Harvest (kg)'].isna()]

    # Prepare training data
    X_train = train_data[features]
    y_train = train_data['Actual Harvest (kg)']

    # Handle missing values in features by filling with median
    X_train.fillna(X_train.median(), inplace=True)

    # Train a regression model (Random Forest for better prediction)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Prepare test data
    X_test = test_data[features]
    X_test.fillna(X_train.median(), inplace=True)

    # Debugging: Check if X_test is empty
    print("X_test shape before prediction:", X_test.shape)

    if not X_test.empty:
        # Predict missing Actual Harvest values
        predicted_actual_harvest = model.predict(X_test)

        # Fill missing values with predicted values (rounded to 2 decimal places)
        farming_data.loc[farming_data['Actual Harvest (kg)'].isna(), 'Actual Harvest (kg)'] = np.round(predicted_actual_harvest, 2)

        # Extract the filled values for reference
        predicted_harvest_df = farming_data.loc[test_data.index, ['Actual Harvest (kg)']]

        # Print predicted values
        print("Predicted Actual Harvest (kg) for Previously Missing Values:")
        print(predicted_harvest_df)
    else:
        print("No valid test samples available for prediction.")

else:
    print("No missing values in 'Actual Harvest (kg)'. No prediction needed.")

In [ ]:
print(farming_data.isnull().sum())

In [ ]:
df_kandy = farming_data[farming_data["Farm Location (District)"] == "Kandy"]

# Get the count and sum of pH values
ph_count = df_kandy["Soil pH Level"].count()
ph_sum = df_kandy["Soil pH Level"].sum()
print(ph_count)
print(ph_sum)

In [ ]:
print("Duplicate rows:",farming_data.duplicated().sum())

In [ ]:
farming_data= farming_data.drop_duplicates()

In [ ]:
print("Duplicate rows:",farming_data.duplicated().sum())


In [ ]:
farming_data.to_csv("cleaned_farmer_data.csv", index=False)


In [ ]:
#calculation for to give score by each creteria trough cleaned_farmer_data.csv
clean_farming_data= pd.read_csv("cleaned_farmer_data.csv")

In [9]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

# Load dataset
my_file_path = r"C:\Users\user\Desktop\iit\year2 sem1\sdgp\AgroEdge\cleaned_farmer_data.csv"
clean_farming_data = pd.read_csv(my_file_path)

# -------------------------------------------
# 🛠 Compute Missing Columns
# -------------------------------------------

# Compute Total Expenses if missing
clean_farming_data['Total Expenses (Rs.)'] = clean_farming_data[['Seed Cost (Rs.)', 'Fertilizer Cost (Rs.)',
                                                                 'Pesticide Cost (Rs.)', 'Irrigation Cost (Rs.)',
                                                                 'Total Labor Wages']].sum(axis=1)

# Compute Total Profit (Rs.)
clean_farming_data['Total Profit (Rs.)'] = clean_farming_data['Total Sales Revenue (Rs.)'] - clean_farming_data['Total Expenses (Rs.)']

# Handle NaN values in profit (if any)
clean_farming_data['Total Profit (Rs.)'].fillna(0, inplace=True)

# -------------------------------------------
# 🛠 Compute Subcategory Scores
# -------------------------------------------

# Yield per Perch
clean_farming_data['Yield per Perch'] = clean_farming_data['Actual Harvest (kg)'] / clean_farming_data['Farm Size (Perches)']
clean_farming_data['Yield Score'] = (clean_farming_data['Yield per Perch'] - clean_farming_data['Yield per Perch'].min()) / \
                                    (clean_farming_data['Yield per Perch'].max() - clean_farming_data['Yield per Perch'].min()) * 100

# Cost per kg
clean_farming_data['Cost per kg'] = clean_farming_data['Total Expenses (Rs.)'] / clean_farming_data['Actual Harvest (kg)']
clean_farming_data['Cost Efficiency Score'] = 100 - np.abs(zscore(clean_farming_data['Cost per kg'])) * 10  

# Fertilizer Efficiency
clean_farming_data['Fertilizer Efficiency'] = clean_farming_data['Actual Harvest (kg)'] / clean_farming_data['Total Fertilizer Quantity (kg)']
clean_farming_data['Fertilizer Score'] = (clean_farming_data['Fertilizer Efficiency'] - clean_farming_data['Fertilizer Efficiency'].min()) / \
                                         (clean_farming_data['Fertilizer Efficiency'].max() - clean_farming_data['Fertilizer Efficiency'].min()) * 100

# Cost Metrics
clean_farming_data['Fertilizer Cost per kg'] = clean_farming_data['Fertilizer Cost (Rs.)'] / clean_farming_data['Actual Harvest (kg)']
clean_farming_data['Pesticide Cost per kg'] = clean_farming_data['Pesticide Cost (Rs.)'] / clean_farming_data['Actual Harvest (kg)']
clean_farming_data['Water Cost per kg'] = clean_farming_data['Irrigation Cost (Rs.)'] / clean_farming_data['Actual Harvest (kg)']
clean_farming_data['Seed Cost per kg'] = clean_farming_data['Seed Cost (Rs.)'] / clean_farming_data['Actual Harvest (kg)']
clean_farming_data['Labor Cost per kg'] = clean_farming_data['Total Labor Wages'] / clean_farming_data['Actual Harvest (kg)']

# Pesticide Efficiency
clean_farming_data['Pesticide Efficiency'] = clean_farming_data['Actual Harvest (kg)'] / clean_farming_data['Total Pesticide Quantity (kg)']
clean_farming_data['Pesticide Score'] = (clean_farming_data['Pesticide Efficiency'] - clean_farming_data['Pesticide Efficiency'].min()) / \
                                        (clean_farming_data['Pesticide Efficiency'].max() - clean_farming_data['Pesticide Efficiency'].min()) * 100

# Water Efficiency
clean_farming_data['Water Efficiency'] = clean_farming_data['Actual Harvest (kg)'] / clean_farming_data['Total Water Usage (liters)']
clean_farming_data['Water Score'] = (clean_farming_data['Water Efficiency'] - clean_farming_data['Water Efficiency'].min()) / \
                                    (clean_farming_data['Water Efficiency'].max() - clean_farming_data['Water Efficiency'].min()) * 100

# Labor Efficiency
clean_farming_data['Labor Efficiency'] = clean_farming_data['Actual Harvest (kg)'] / clean_farming_data['Total Number of Labor Hours']
clean_farming_data['Labor Score'] = (clean_farming_data['Labor Efficiency'] - clean_farming_data['Labor Efficiency'].min()) / \
                                    (clean_farming_data['Labor Efficiency'].max() - clean_farming_data['Labor Efficiency'].min()) * 100

# Soil Suitability Score
clean_farming_data['Soil Suitability Score'] = 100 * np.exp(-1.5 * np.abs(clean_farming_data['Soil pH Level'] - 6.2))

# Seed Efficiency
clean_farming_data['Seed Efficiency'] = clean_farming_data['Actual Harvest (kg)'] / clean_farming_data['Seed Quantity (g)']
clean_farming_data['Seed Efficiency Score'] = (clean_farming_data['Seed Efficiency'] - clean_farming_data['Seed Efficiency'].min()) / \
                                              (clean_farming_data['Seed Efficiency'].max() - clean_farming_data['Seed Efficiency'].min()) * 100

# Market Success
market_avg_price = clean_farming_data['Market Selling Price (Rs./kg)'].mean()
clean_farming_data['Market Price Score'] = (clean_farming_data['Market Selling Price (Rs./kg)'] / market_avg_price) * 100

clean_farming_data['Total Sales Revenue per Perch'] = clean_farming_data['Total Sales Revenue (Rs.)'] / clean_farming_data['Farm Size (Perches)']
best_revenue_per_perch = clean_farming_data['Total Sales Revenue per Perch'].max()
clean_farming_data['Revenue Score'] = (clean_farming_data['Total Sales Revenue per Perch'] / best_revenue_per_perch) * 100

# ROI Score (Min-Max Normalization)
clean_farming_data['ROI Score'] = (clean_farming_data['Total Profit (Rs.)'] - clean_farming_data['Total Profit (Rs.)'].min()) / \
                                  (clean_farming_data['Total Profit (Rs.)'].max() - clean_farming_data['Total Profit (Rs.)'].min()) * 100

# -------------------------------------------
# 🎯 Compute Weighted Scores for Final Score
# -------------------------------------------
weights = {
    'Yield Score': 0.25,
    'Cost Efficiency Score': 0.10,
    'Fertilizer Score': 0.05,
    'Fertilizer Cost per kg': 0.02,
    'Pesticide Score': 0.05,
    'Pesticide Cost per kg': 0.02,
    'Water Score': 0.05,
    'Water Cost per kg': 0.02,
    'Labor Score': 0.05,
    'Labor Cost per kg': 0.02,
    'Seed Efficiency Score': 0.05,
    'Seed Cost per kg': 0.02,
    'Soil Suitability Score': 0.20,
    'Market Price Score': 0.15,
    'Revenue Score': 0.10,
    'ROI Score': 0.05
}

clean_farming_data['Final Farmer Score'] = sum(
    clean_farming_data[col] * weight for col, weight in weights.items()
)

clean_farming_data = clean_farming_data.round(2)

# -------------------------------------------
# 🔍 Display Farmer's Report + Top 3 Farmers
# -------------------------------------------
def display_farmer_report(farm_id):
    farmer = clean_farming_data[clean_farming_data['Farm ID'] == farm_id].iloc[0]
    print(f"\n🚜 Farmer Performance Report for Farm ID: {farm_id}")
    print("-" * 50)
    for metric, weight in weights.items():
        print(f"- {metric}: {farmer[metric]} / 100  →  ({round(farmer[metric] * weight, 2)} Marks)")
    print(f"\n🏆 **Final Farmer Score: {farmer['Final Farmer Score']} / 100**")

    # Display Top 3 Farmers
    top_farmers = clean_farming_data.nlargest(3, 'Final Farmer Score')[['Farm ID', 'Final Farmer Score']]
    print("\n🔥 **Top 3 Farmers:**")
    for i, row in top_farmers.iterrows():
        print(f"Farm ID: {row['Farm ID']} - Score: {row['Final Farmer Score']}")

# Call function for a specific farm ID
display_farmer_report(6710)



🚜 Farmer Performance Report for Farm ID: 6710
--------------------------------------------------
- Yield Score: 27.76 / 100  →  (6.94 Marks)
- Cost Efficiency Score: 62.67 / 100  →  (6.27 Marks)
- Fertilizer Score: 0.23 / 100  →  (0.01 Marks)
- Fertilizer Cost per kg: 137.58 / 100  →  (2.75 Marks)
- Pesticide Score: 0.42 / 100  →  (0.02 Marks)
- Pesticide Cost per kg: 22.81 / 100  →  (0.46 Marks)
- Water Score: 74.15 / 100  →  (3.71 Marks)
- Water Cost per kg: 63.74 / 100  →  (1.27 Marks)
- Labor Score: 0.11 / 100  →  (0.01 Marks)
- Labor Cost per kg: 38.66 / 100  →  (0.77 Marks)
- Seed Efficiency Score: 100.0 / 100  →  (5.0 Marks)
- Seed Cost per kg: 43.94 / 100  →  (0.88 Marks)
- Soil Suitability Score: 16.53 / 100  →  (3.31 Marks)
- Market Price Score: 48.31 / 100  →  (7.25 Marks)
- Revenue Score: 7.95 / 100  →  (0.8 Marks)
- ROI Score: 5.84 / 100  →  (0.29 Marks)

🏆 **Final Farmer Score: 39.73 / 100**

🔥 **Top 3 Farmers:**
Farm ID: 6503.0 - Score: 90.37
Farm ID: 9428.0 - Score: 88